In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = '/content/drive/MyDrive/생성 AI 모델링/data/'
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
import csv

In [4]:
train_yogiyo = pd.read_csv(f'{DATA_PATH}yogiyo_reviews_jsi_all.csv')
# train_playstore = pd.read_csv(f'{DATA_PATH}playstore_df.csv')

In [5]:
new_columns = ['review','owner_reply']

cols = ['고객리뷰','사장댓글']
yogiyo = train_yogiyo[cols]
yogiyo.columns=new_columns



In [6]:
yogiyo.shape

(18686, 2)

In [7]:
yogiyo['review'] = yogiyo['review'].str.replace("[^a-zA-Z가-힣0-9 .,!?\'\"]" , "",regex=True) #포함 안된것들은 다 지우기
yogiyo['owner_reply'] = yogiyo['owner_reply'].str.replace("[^a-zA-Z가-힣0-9 .,!?\'\"]" , "",regex=True)


shuffled_df = yogiyo.sample(frac=1, random_state=42)  # frac=1은 모든 행을 선택, random_state는 재현 가능한 랜덤 설정

sample_df = shuffled_df.iloc[:1000]


yogiyo.shape, shuffled_df.shape, sample_df.shape

<ipython-input-7-a124fad2a441>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yogiyo['review'] = yogiyo['review'].str.replace("[^a-zA-Z가-힣0-9 .,!?\'\"]" , "",regex=True) #포함 안된것들은 다 지우기
<ipython-input-7-a124fad2a441>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yogiyo['owner_reply'] = yogiyo['owner_reply'].str.replace("[^a-zA-Z가-힣0-9 .,!?\'\"]" , "",regex=True)


((18686, 2), (18686, 2), (1000, 2))

In [8]:
shuffled_df

,review,owner_reply
16031,너무맛있습니다!!!,황금별5개 감사드립니다소중한 리뷰감사드리고 행복한 나날되세요피자는 피제이피자3호점고...
8211,주말에 밥 하기 귀찮지만 든든히 먹고 싶어서 시켰어요!너무 바삭하고 맛있습니다 치즈...,rk님 주말에 밥하기는 귀찮지만 든든히 먹고 싶어서 시키셨다니 격하게 공감되네요 ...
17900,완전 굿 최소주문가격도 낮고 배달비도 낮고 완전 짱임 음식 맛도 짱맛있음 오늘 츄러...,저희 킹프레소를 이용해주셔서 감사합니다.고객님께 만족을 드릴 수 있어 기쁩니다. ...
17530,Verry good,맛있게 드셨는지요 진심어린 리뷰 별이5개 고맙습니다 당일 삶은 족발보쌈 해장국감자탕...
13135,가격대비 좀 심함,돈코츠세트가 제일인기많은 메뉴인데 많이 부족했나봅니다 죄송합니다
...,...,...
11284,"토핑많고 맛있네요,",안녕하세요 탐나는피자 부평점 입니다 D소중한 리뷰와 별점 너무 감사드립니다.피자가 ...
11964,우앗 맛나게 잘 먹었습니다!포장도 깔끔하고 닭발양도 많아서 만족했습니다!!,항상 맛과 청결에 정성을 다하겠습니다주문해주셔서 감사합니다
5390,종종 시켜먹는데 맛있어요!!!,저희 올데이파스타 동대문구점을 찾아주셔서 정말 대단히 감사합니다이렇게 맛있게 드셔주...
860,맛있게 잘 먹었습니다,안녕하세요 고돼지 송파점입니다별점만점 그리고 따뜻한 리뷰 작성해주셔서 감사합니다. ...


# 전처리 (문장추출)


## 문장 추출

### review

- 문장단위로 분리 (.,?,!) 이외에도 다양한 구분기호를 활용


In [9]:
text = yogiyo['review']
text

0           사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요
1                        리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!
2                      잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.
3                                      굿 말이 필요 없는 메뉴 였습니다
4                                             맛있어요 양도많구좋음
                               ...                       
18681                                          맛있게 잘 먹었어요
18682           요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다
18683                                배달 빨라요 맛있게 잘 먹었습니당 !
18684                                              잘 먹었어요
18685    식후감 자주 시키는데 항상 요청사항도 잘들어주시고 맛있고 빨라요 앞으로도 자주 시킬게요
Name: review, Length: 18686, dtype: object

In [10]:
import re

# 패턴을 정의합니다. 여러 가지 구분 기호와 줄바꿈 문자를 포함합니다.
sentence_pattern = re.compile(r'[^.!?]*[.!?]|[^.!?]+$') # $로 구분기호 없이 끝나는 경우 포함   # 구분기호 없는 경우 포함 문장 분리 버전. 이 버전 or 아래 간단한 버전 쓰면될 듯

yogiyo_reviews_sentence = []

for review in text:  # text는 문자열 리뷰들의 리스트입니다
    if isinstance(review, str):  # 리뷰가 문자열인지 확인합니다
        sentences = sentence_pattern.findall(review)
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        yogiyo_reviews_sentence.append(sentences)
    else:
        review = '리뷰없음'
        yogiyo_reviews_sentence.append(review) # 리뷰 없는 경우도 갯수, 순서 맞추기 위해 넣어줌
        print(review)



In [11]:
len(yogiyo_reviews_sentence), yogiyo.shape

(18686, (18686, 2))

In [12]:
yogiyo_reviews_sentence

[['사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요'],
 ['리뷰가 늦었네요.', '.', '맛있게 잘 먹었습니다.', '또 시킬게요!', '!'],
 ['잘 먹었습니다.', '다만 주문에 있던 펩시콜라는 받지 못했습니다.'],
 ['굿 말이 필요 없는 메뉴 였습니다'],
 ['맛있어요 양도많구좋음'],
 ['양많고 맛있었어요'],
 ['처음 주문했는데 맛있어서 다먹었어요 리뷰 묵사발도 꼭 드세요!', '맛있게 잘먹었습니다 다음에 또 주문할게요'],
 ['가성비 아주 만족 합니다.', '맛있게 잘먹었습니다.'],
 ['맛있게 먹었습니다!', '!'],
 ['새벽부터 시켰는데 배송 시간 맞춰서 잘 와주셨고도시락도 맛있고 묵사발도 맛있습니다!', '!', '!'],
 ['정말 간만에 리뷰이벤트 받았기 때문이 아닌 진짜 맛있어서 리뷰써요'],
 ['맛있게 잘 먹었습니다.'],
 ['처음 주문해봤는데 엄청 푸짐하게 잘먹었습니다 감사합니다'],
 ['맛있게 잘 먹었습니다!'],
 ['맛있게 잘먹었습니다'],
 ['비비는게?', '힘들정도로 양도 엄청 많아요!', '양 뿐만아니라 맛도 진짜 맛잇어요 완전 추천!'],
 ['저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요.', '.', '된장찌개도 엄청 짜고 그래도 맛은 있었어요'],
 ['진짜 너무 자주 시켜먹어욧 소스가 삸싹 시큼한게 넘나 맛있고 고기도두툼하고  계란후라이 추가 사이드있으면 좋을거같아요'],
 ['김치찌개가 좀 매워요 맛있게 잘먹었습니다'],
 ['친구시켜줬는데 사진은 이렇지만 .', '.', '고기 많고 맛있대요 !', '!', '!', '!', '감사합니다'],
 ['혼자먹기 좋아요 긋'],
 ['맛있어요 또시킬게용'],
 ['맛있게 잘먹었습니다 양도 많아요'],
 ['20분 이내 배달 알림 주시고는 한시간이 지나서야 받았습니다.',
  '배달이 늦는 점에 대한 고지가 없어 전화드리니 이유는 말씀 없이 배달기사분이 픽업했다는 답만 있었구요.',
  

- ... / ??? / ?! 등 구분기호 분리 시에 두개 이상인 경우 문장으로 인식하는 것 제거 처리

In [13]:
filtered_reviews = []

for review in yogiyo_reviews_sentence:
    filtered_review = []
    for sentence in review:
        if sentence not in ['.', '?', '!']:
            filtered_review.append(sentence)
    filtered_reviews.append(filtered_review)

yogiyo_reviews_sentence = filtered_reviews


In [14]:
len(yogiyo_reviews_sentence)

18686

In [15]:
yogiyo_reviews_sentence

[['사진보다 실물이 백배 맛있어요 고기 양도 많아요 배달시간도 딱 맞춰서 보내주셨어요'],
 ['리뷰가 늦었네요.', '맛있게 잘 먹었습니다.', '또 시킬게요!'],
 ['잘 먹었습니다.', '다만 주문에 있던 펩시콜라는 받지 못했습니다.'],
 ['굿 말이 필요 없는 메뉴 였습니다'],
 ['맛있어요 양도많구좋음'],
 ['양많고 맛있었어요'],
 ['처음 주문했는데 맛있어서 다먹었어요 리뷰 묵사발도 꼭 드세요!', '맛있게 잘먹었습니다 다음에 또 주문할게요'],
 ['가성비 아주 만족 합니다.', '맛있게 잘먹었습니다.'],
 ['맛있게 먹었습니다!'],
 ['새벽부터 시켰는데 배송 시간 맞춰서 잘 와주셨고도시락도 맛있고 묵사발도 맛있습니다!'],
 ['정말 간만에 리뷰이벤트 받았기 때문이 아닌 진짜 맛있어서 리뷰써요'],
 ['맛있게 잘 먹었습니다.'],
 ['처음 주문해봤는데 엄청 푸짐하게 잘먹었습니다 감사합니다'],
 ['맛있게 잘 먹었습니다!'],
 ['맛있게 잘먹었습니다'],
 ['비비는게?', '힘들정도로 양도 엄청 많아요!', '양 뿐만아니라 맛도 진짜 맛잇어요 완전 추천!'],
 ['저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요.', '된장찌개도 엄청 짜고 그래도 맛은 있었어요'],
 ['진짜 너무 자주 시켜먹어욧 소스가 삸싹 시큼한게 넘나 맛있고 고기도두툼하고  계란후라이 추가 사이드있으면 좋을거같아요'],
 ['김치찌개가 좀 매워요 맛있게 잘먹었습니다'],
 ['친구시켜줬는데 사진은 이렇지만 .', '고기 많고 맛있대요 !', '감사합니다'],
 ['혼자먹기 좋아요 긋'],
 ['맛있어요 또시킬게용'],
 ['맛있게 잘먹었습니다 양도 많아요'],
 ['20분 이내 배달 알림 주시고는 한시간이 지나서야 받았습니다.',
  '배달이 늦는 점에 대한 고지가 없어 전화드리니 이유는 말씀 없이 배달기사분이 픽업했다는 답만 있었구요.',
  '제육 시켰는데 갈비도시락이 오고, 김치찌개는 밍밍하고 식어있어 많이 아쉬웠습니다 다른 메

### owner_reply

In [16]:
text = yogiyo['owner_reply']
text

0        이렇게 저희 매장 찾아주셔서 감사드립니다!!맛 뿐만 아니라 양도, 서비스도 모두 최...
1        좋은 리뷰 덕분에 기분이 행복해지는 것 같아요고객님께서 맛있다고 말씀 주시니 피로가...
2        소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3        수 많은 매장들을 헤치고 와주셔서 감사합니다!!저희 매장의 메뉴를 제대로 즐기신 거...
4        수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 ...
                               ...                        
18681                                감사합니다!!! 다음에도 이용해주세용 
18682                                죄송합니다 !! 다음에도 애용해주세용 
18683                                감사합니다!!! 다음에도 이용해주세용 
18684                                감사합니다!!! 다음에도 이용해주세용 
18685                                감사합니다!!! 다음에도 이용해주세용 
Name: owner_reply, Length: 18686, dtype: object

- 문장단위로 분리 (.,?,!) 이외에도 다양한 구분기호를 활용


In [17]:
import re

# 패턴을 정의합니다. 여러 가지 구분 기호와 줄바꿈 문자를 포함합니다.
sentence_pattern = re.compile(r'[^.!?]*[.!?]|[^.!?]+$') # $로 구분기호 없이 끝나는 경우 포함   # 구분기호 없는 경우 포함 문장 분리 버전. 이 버전 or 아래 간단한 버전 쓰면될 듯

yogiyo_reply_sentence = []

for review in text:  # text는 문자열 리뷰들의 리스트입니다
    if isinstance(review, str):  # 리뷰가 문자열인지 확인합니다
        sentences = sentence_pattern.findall(review)
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        yogiyo_reply_sentence.append(sentences)
    else:
        review = '리뷰없음'
        yogiyo_reply_sentence.append(review) # 리뷰 없는 경우도 갯수, 순서 맞추기 위해 넣어줌
        print(review)



In [18]:
len(yogiyo_reply_sentence), yogiyo.shape

(18686, (18686, 2))

In [19]:
yogiyo_reply_sentence

[['이렇게 저희 매장 찾아주셔서 감사드립니다!',
  '!',
  '맛 뿐만 아니라 양도, 서비스도 모두 최상의 품질을 드리려고 노력하고 있습니다 모든 직원들이 힘내서 고객님의 마음을 잡기 위해 계속 노력하겠습니다항상 건강 챙기시고, 오늘도 즐거운 하루 보내시길 바라겠습니다'],
 ['좋은 리뷰 덕분에 기분이 행복해지는 것 같아요고객님께서 맛있다고 말씀 주시니 피로가 싹 사라지네요!',
  '주방에서 힘든 일도 있고 늘 쉽지만은 않지만, 이런 리뷰 읽을 때 마다 보람차고 다시 시작할 수 있는 거 같습니다 정말 감사드려요오늘도 화이팅 있게 마무리하시길 바라요'],
 ['소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다.',
  '저희를 찾아 주신 만큼 100 만족감을 드리고 싶었는데 그러지 못한 것 같아 너무 죄송합니다 말씀 주신 부분에 대해서는 주의하여 다음 주문시에는 실수 없도록 꼼꼼하게 확인하겠습니다.',
  '.',
  '!',
  '!',
  '저희 매장을 찾아 주셔서 다시 한번 감사의 말씀드리며, 언제나 행복한 순간들만 가득하시길 바라겠습니다!'],
 ['수 많은 매장들을 헤치고 와주셔서 감사합니다!',
  '!',
  '저희 매장의 메뉴를 제대로 즐기신 거 같아 너무 영광이에요 모두 마음에 드셨을까요?',
  '?',
  '전부 정성껏 준비했지만, 늘 주문해 주신 분들의 평가를 기다릴 땐 떨려요더욱 만족스러운 시간 드릴 수 있게 노력하겠습니다'],
 ['수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 성공이였을까요?',
  '?',
  '저희 메뉴 칭찬해 주셔서 감사합니다 고객님!',
  '!',
  '늘 어떻게하면 더 맛있게 드릴 수 있을까 고민 많이 하고 있어요항상 노력하고 발전하도록 하겠습니다항상 같은 만족감 드릴 수 있도록 언제나 최선을 다해 운영 하겠습니다'],
 ['저희 매장 찾아주신 것도 감사한데 리뷰까지.',
  '.',
  '감동입니다고객님들의 배고픔을 저희는 지켜만 볼 수 없습니다!',
 

- ... / ??? / ?! 등 구분기호 분리 시에 두개 이상인 경우 문장으로 인식하는 것 제거 처리

In [20]:
filtered_reviews = []

for review in yogiyo_reply_sentence:
    filtered_review = []
    for sentence in review:
        if sentence not in ['.', '?', '!']:
            filtered_review.append(sentence)
    filtered_reviews.append(filtered_review)

yogiyo_reply_sentence = filtered_reviews


In [21]:
len(yogiyo_reply_sentence)

18686

In [22]:
yogiyo_reply_sentence

[['이렇게 저희 매장 찾아주셔서 감사드립니다!',
  '맛 뿐만 아니라 양도, 서비스도 모두 최상의 품질을 드리려고 노력하고 있습니다 모든 직원들이 힘내서 고객님의 마음을 잡기 위해 계속 노력하겠습니다항상 건강 챙기시고, 오늘도 즐거운 하루 보내시길 바라겠습니다'],
 ['좋은 리뷰 덕분에 기분이 행복해지는 것 같아요고객님께서 맛있다고 말씀 주시니 피로가 싹 사라지네요!',
  '주방에서 힘든 일도 있고 늘 쉽지만은 않지만, 이런 리뷰 읽을 때 마다 보람차고 다시 시작할 수 있는 거 같습니다 정말 감사드려요오늘도 화이팅 있게 마무리하시길 바라요'],
 ['소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다.',
  '저희를 찾아 주신 만큼 100 만족감을 드리고 싶었는데 그러지 못한 것 같아 너무 죄송합니다 말씀 주신 부분에 대해서는 주의하여 다음 주문시에는 실수 없도록 꼼꼼하게 확인하겠습니다.',
  '저희 매장을 찾아 주셔서 다시 한번 감사의 말씀드리며, 언제나 행복한 순간들만 가득하시길 바라겠습니다!'],
 ['수 많은 매장들을 헤치고 와주셔서 감사합니다!',
  '저희 매장의 메뉴를 제대로 즐기신 거 같아 너무 영광이에요 모두 마음에 드셨을까요?',
  '전부 정성껏 준비했지만, 늘 주문해 주신 분들의 평가를 기다릴 땐 떨려요더욱 만족스러운 시간 드릴 수 있게 노력하겠습니다'],
 ['수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 성공이였을까요?',
  '저희 메뉴 칭찬해 주셔서 감사합니다 고객님!',
  '늘 어떻게하면 더 맛있게 드릴 수 있을까 고민 많이 하고 있어요항상 노력하고 발전하도록 하겠습니다항상 같은 만족감 드릴 수 있도록 언제나 최선을 다해 운영 하겠습니다'],
 ['저희 매장 찾아주신 것도 감사한데 리뷰까지.',
  '감동입니다고객님들의 배고픔을 저희는 지켜만 볼 수 없습니다!',
  '언제든지 행복한 배부름을 느끼게 해 드릴 수 있도록 저희가 있지 않겠습니까!',
  '매번 찾아 주셔도 매번 

## 특정 문장 추출

### 업체 관련 문장 (store, region)

- 1) 고유명사 업체명이 들어간 문장을 모두 삭제
- 2) 고유명사 업체명을 일반화 명사로 변환 ('봄봄카페' -> '저희 카페', '미친피자' -> '저희 피자집'
- 3) 고유명사 업체명을 store로 학습, input데이터로 넣어 따로 학습하도록 만들기

In [23]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 56.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602628 sha256=894d853846e86a189ae10daac205fd50c25f1f0d420f420bf4a602b6578e2446
  Stored in directory: /root/.cache/pip/wheels/f3/55/41/ca474338ece1bc4314b01445f64ff002d71e19df45575a16e2
Successfully built kiwipiepy-model


In [24]:
total_reply_sentence = []

for review in yogiyo_reply_sentence:
  for sentence in review :
    total_reply_sentence.append(sentence)

In [25]:
len(total_reply_sentence) ## 5.6만개의 문장

56141

In [26]:
from kiwipiepy import Kiwi

# Kiwi 객체 생성
kiwi = Kiwi()

kiwi.prepare()


- 어떤 고유명사들 추출되는지 nnp_list로 확인

In [27]:
nnp_list = []

# 문장들을 순회하며 고유명사(사람 이름)를 추출
for review in yogiyo_reply_sentence:
    for sentence in review:
        tokens = kiwi.analyze(sentence)
        proper_nouns = [token[0] for token in tokens[0][0] if token[1] == 'NNP']

        # 중복 제거하면서 nnp_list에 추가
        for noun in proper_nouns:
            if noun not in nnp_list:
                nnp_list.append(noun)


- 요기요 고유명사의 경우 사람 아닌 메뉴명 위주 / 업체명도 별로 없음

In [28]:
nnp_list

['안녕하세요',
 '응답하라',
 '행복합니다',
 '라',
 '미생',
 '완생',
 '아메리카노',
 '와플',
 '크로넛',
 '다쿠아즈',
 '슈톨렌',
 '워커스',
 '필살기',
 '캔아메',
 '석촌',
 '상콤',
 '글레이즈드',
 '수저포크',
 '싱',
 '크로넛은',
 '벨기에',
 '리에주',
 '레이즈',
 '바닐라',
 '디게싱',
 '플로',
 '워커스커피로스터스입니다저희는',
 '홀',
 '라이더',
 '요기요',
 '초코',
 '알랭',
 '티라미수',
 '워커스커피',
 '뺑오플',
 '로스터스',
 '그라인더',
 '라떼',
 '송파',
 '샤방샤',
 '오리진',
 '반갑습니다',
 '쿠팡',
 '로제',
 '크림',
 '네넵',
 '문정',
 '떡깨비',
 '배',
 '헬리',
 '뉴욕',
 '탄',
 '정해진',
 '한강',
 '당치땡이',
 '사이드',
 '솔트',
 '헬리오시티',
 '당치땡',
 '하트',
 '서울시',
 '띠용',
 '힝구리',
 '정',
 '하튜저희',
 '벨',
 '오',
 '너어어어무',
 '기쁘네용다',
 '용저',
 '청양고추',
 '일상다반사',
 '진',
 '갑인',
 '지연',
 '해주세용저희',
 '수육국밥',
 '감사합니',
 '따봉',
 '한',
 '우',
 '아디오스저희',
 '찬',
 '다대기',
 '행쇼',
 '인도',
 '행쇼저희',
 '행복하세요요요용아디오스저희',
 '송파구',
 '부산',
 '나이스',
 '양',
 '채운',
 '자반고등어',
 '찐',
 '이모티콘',
 '아디오스',
 '고객님아디오스저희',
 '러버',
 '굿아디오스저희',
 '봬요',
 '아자아자',
 '백',
 '마라',
 '손수경',
 '김',
 '고마워요',
 '무국적',
 '김치볶음밥',
 '스팸',
 '치즈볼',
 '청',
 '이승철',
 '만다린',
 '오예',
 '녕하세요',
 '바운스',
 '팟타이',
 '닭강정',
 '우왕',
 '무한',
 '데넘',
 '드',
 '커큐민',


- 실제 업체명 이름 리스트를 활용해서, 이름 들어가는 문장으로 추출하기

In [29]:
ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]
len(ygy_store_lst)

125

In [30]:
store_name = []
region_name = []

for store in ygy_store_lst :
  if '-' in store:
    item = store.split('-')[0]
    item2 = store.split('-')[1]
    store_name.append(item)
    if item2[-1] == '점':
      region_name.append(item2)

  else:
    store_name.append(store)


In [31]:
store_name

['1988응답하라추억의옛날도시락',
 '워커스커피 로스터스',
 '떡깨비',
 '당치땡',
 '우시장국밥',
 '고돼지',
 '마라의신마라탕&마라샹궈',
 '춘리마라탕',
 '손수경의육회담은연어',
 '무국적식탁',
 '청년치킨',
 '송파만다린',
 '배달의쌀국수',
 '닭강점',
 '토핑폭탄김치찜&김치찌개',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지',
 '잭아저씨족발보쌈',
 '육회&연어',
 '갈비민족',
 '구구족',
 '피자보이시나',
 '왕빈자삼파전',
 '행복한찜닭',
 '스시사소우',
 '저팔계&중국&마라탕&덮밥&요리&전문점',
 '명작파스타',
 '올데이파스타',
 '대한냉면',
 '인생제육',
 '고기듬뿍미트박스',
 '피자스웨그',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다',
 '더팔당매운오징어&닭발 김치찜',
 '우리집반찬도시락',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비&마약볶음밥',
 '대찌 대파생고기김치찌개&정육왕국물두루치기',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄김치찜&김치찌개',
 '1989마라탕',
 '미친피자',
 '천년닭강정',
 '수상한삼겹살',
 '수제죽전문점',
 '오늘은 분식',
 '이끌림마라탕',
 '탐나는피자',
 '고기혁명',
 '라화방마라탕',
 '전금례닭볶음탕',
 '큰아들백순대',
 '공주닭발',
 '신자매김치찜&김치찌개',
 '짱닭치킨',
 '이태리면가게',
 '폴트버거',
 '랜돌프뉴욕페페로니',
 '홍싸롱',
 '쇼부덮밥',
 '카산도',
 '부성초밥',
 '닭장수섭삼계탕&전

In [32]:
region_name

['방이점',
 '가락점',
 '헬리오시티점',
 '석촌점',
 '송파점',
 '송파점',
 '가락점',
 '송파점',
 '삼전점',
 '송파점',
 '헬리오시티점',
 '송파점',
 '송파점',
 '본점',
 '미아본점',
 '본점',
 '성신여대역점',
 '대학로점',
 '미아본점',
 '성북점',
 '성북점',
 '동대문구점',
 '성북점',
 '성북점',
 '성북점',
 '북서울꿈의숲점',
 '혜화막내딸지점',
 '부천점',
 '부천점',
 '부평점',
 '부천부평점',
 '부천점',
 '부평점',
 '부천점',
 '신중동점',
 '부천점',
 '부천시청점',
 '도촌점',
 '야탑점',
 '판교점',
 '야탑점',
 '정자본점',
 '미금본점',
 '미금점',
 '분당정자점',
 '분당미금역점',
 '죽전점',
 '용인수지점',
 '서현본점',
 '분당점']

- 메뉴 이름들의 pool : 리스트. 이 리스트를 삭제한 고유 업체명이 들어가는 문장들에 대해서 확인해보기

In [33]:
menu_name = ['도시락','카레','커피', '국밥','마라탕','마라샹궈','연어','치킨','쌀국수','닭강정','김치찜','김치찌개',
                '족발','보쌈','육회','갈비','피자','찜닭','파스타','냉면','곱창','닭발','반찬','곱도리탕','순대','떡볶이','부대찌개'
                '삼겹살','분식','라면','버거','덮밥','초밥','통닭','짜장','카페','와플','누들']

In [34]:
store_nnp = []

for store in store_name:
    found = False
    for menu in menu_name:
        if menu in store:
            found = True
            store = store.replace(menu, "").strip()  # 가게명에서 메뉴명 제거
            break
    if not found:
        store_nnp.append(store)
    else:
        store_nnp.append(store)


In [35]:
len(store_nnp)

125

In [90]:
store_nnp

['1988응답하라추억의옛날',
 '워커스 로스터스',
 '떡깨비',
 '당치땡',
 '우시장',
 '고돼지',
 '마라의신&마라샹궈',
 '춘리',
 '손수경의육회담은',
 '무국적식탁',
 '청년',
 '송파만다린',
 '배달의',
 '닭강점',
 '토핑폭탄&김치찌개',
 '조형훈보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지',
 '잭아저씨보쌈',
 '육회&',
 '민족',
 '구구족',
 '보이시나',
 '왕빈자삼파전',
 '행복한',
 '스시사소우',
 '저팔계&중국&&덮밥&요리&전문점',
 '명작',
 '올데이',
 '대한',
 '인생제육',
 '고기듬뿍미트박스',
 '스웨그',
 '청춘식당묵은지',
 '삽교원조두리',
 '뜸들이다',
 '더팔당매운오징어&닭발',
 '우리집반찬',
 '마라쿡',
 '김포불',
 '헬로팬닭&마약볶음밥',
 '대찌 대파생고기&정육왕국물두루치기',
 '구래상회',
 '승도리네',
 '자성당',
 '수락',
 '나인',
 '어시장',
 '돈까스회관',
 '호랑이',
 '땡초막창',
 '이탈리',
 '텐텐',
 '과일에반하다',
 '삼다수',
 '병천토속',
 '최고남제육명가',
 '짱닭',
 '걸작',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄&김치찌개',
 '1989',
 '미친',
 '천년',
 '수상한삼겹살',
 '수제죽전문점',
 '오늘은',
 '이끌림',
 '탐나는',
 '고기혁명',
 '라화방',
 '전금례닭볶음탕',
 '큰아들백',
 '공주',
 '신자매&김치찌개',
 '짱닭',
 '이태리면가게',
 '폴트',
 '랜돌프뉴욕페페로니',
 '홍싸롱',
 '쇼부',
 '카산도',
 '부성',
 '닭장수섭삼계탕&전기구이',
 '깨돌이김밥',
 '울진죽변항',
 'GTS',
 '코브라독스',
 '마라하오',
 '땅끝수산',
 '대통령',
 '올데이케밥&샐러드',
 '야키토리 카마쿠라',
 '비오키친',
 '부산오빠1인',
 '알촌',
 '고씨네',
 '

In [91]:
for i in store_nnp:
  if len(i) < 3 :
    store_nnp.remove(i)

len(store_nnp)

103

In [92]:
store_nnp

['1988응답하라추억의옛날',
 '워커스 로스터스',
 '떡깨비',
 '당치땡',
 '우시장',
 '고돼지',
 '마라의신&마라샹궈',
 '손수경의육회담은',
 '무국적식탁',
 '송파만다린',
 '배달의',
 '닭강점',
 '토핑폭탄&김치찌개',
 '조형훈보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지',
 '잭아저씨보쌈',
 '육회&',
 '구구족',
 '보이시나',
 '왕빈자삼파전',
 '행복한',
 '스시사소우',
 '저팔계&중국&&덮밥&요리&전문점',
 '올데이',
 '인생제육',
 '고기듬뿍미트박스',
 '스웨그',
 '청춘식당묵은지',
 '삽교원조두리',
 '뜸들이다',
 '더팔당매운오징어&닭발',
 '우리집반찬',
 '마라쿡',
 '김포불',
 '헬로팬닭&마약볶음밥',
 '대찌 대파생고기&정육왕국물두루치기',
 '구래상회',
 '승도리네',
 '자성당',
 '나인',
 '어시장',
 '돈까스회관',
 '호랑이',
 '땡초막창',
 '이탈리',
 '과일에반하다',
 '삼다수',
 '병천토속',
 '최고남제육명가',
 '걸작',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄&김치찌개',
 '1989',
 '천년',
 '수상한삼겹살',
 '수제죽전문점',
 '오늘은',
 '이끌림',
 '탐나는',
 '고기혁명',
 '라화방',
 '전금례닭볶음탕',
 '큰아들백',
 '신자매&김치찌개',
 '이태리면가게',
 '랜돌프뉴욕페페로니',
 '홍싸롱',
 '카산도',
 '닭장수섭삼계탕&전기구이',
 '깨돌이김밥',
 '울진죽변항',
 'GTS',
 '코브라독스',
 '마라하오',
 '땅끝수산',
 '대통령',
 '올데이케밥&샐러드',
 '야키토리 카마쿠라',
 '비오키친',
 '부산오빠1인',
 '고씨네',
 '뜸들이다',
 '짚신스시',
 '39',
 '피제이',
 '황궁쟁반',
 'OK반점',
 '모모타코야키&',
 '꼬알라파이',
 '프라텔로',
 '한식세끼1인&김치찌개',
 '윤희횟집',


In [93]:
store_name + store_nnp + region_name

['1988응답하라추억의옛날도시락',
 '워커스커피 로스터스',
 '떡깨비',
 '당치땡',
 '우시장국밥',
 '고돼지',
 '마라의신마라탕&마라샹궈',
 '춘리마라탕',
 '손수경의육회담은연어',
 '무국적식탁',
 '청년치킨',
 '송파만다린',
 '배달의쌀국수',
 '닭강점',
 '토핑폭탄김치찜&김치찌개',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지',
 '잭아저씨족발보쌈',
 '육회&연어',
 '갈비민족',
 '구구족',
 '피자보이시나',
 '왕빈자삼파전',
 '행복한찜닭',
 '스시사소우',
 '저팔계&중국&마라탕&덮밥&요리&전문점',
 '명작파스타',
 '올데이파스타',
 '대한냉면',
 '인생제육',
 '고기듬뿍미트박스',
 '피자스웨그',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다',
 '더팔당매운오징어&닭발 김치찜',
 '우리집반찬도시락',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비&마약볶음밥',
 '대찌 대파생고기김치찌개&정육왕국물두루치기',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄김치찜&김치찌개',
 '1989마라탕',
 '미친피자',
 '천년닭강정',
 '수상한삼겹살',
 '수제죽전문점',
 '오늘은 분식',
 '이끌림마라탕',
 '탐나는피자',
 '고기혁명',
 '라화방마라탕',
 '전금례닭볶음탕',
 '큰아들백순대',
 '공주닭발',
 '신자매김치찜&김치찌개',
 '짱닭치킨',
 '이태리면가게',
 '폴트버거',
 '랜돌프뉴욕페페로니',
 '홍싸롱',
 '쇼부덮밥',
 '카산도',
 '부성초밥',
 '닭장수섭삼계탕&전

In [94]:
store_sentence = []    # store_name 혹은 region_name 리스트 단어를 포함하는 문장들

for review in yogiyo_reply_sentence:
    for sentence in review:
      if any(keyword in sentence for keyword in (store_name + store_nnp + region_name)): #store nnp보다 store name이 나음
        store_sentence.append(sentence)


In [95]:
store_sentence

['저희 매장을 찾아 주셔서 다시 한번 감사의 말씀드리며, 언제나 행복한 순간들만 가득하시길 바라겠습니다!',
 '언제든지 행복한 배부름을 느끼게 해 드릴 수 있도록 저희가 있지 않겠습니까!',
 '언제나 웃음 가득한 일들만 계속되시길 바라며, 행복한 하루 보내세요',
 '앞으로도 맛의 성숙도를 높이기 위해 부던히 노력하겠습니다앞으로도 행복한 하루 보내시길 바랄게요',
 '저희 1988응답하라추억의옛날도시락의 메뉴를 주문해 주셔서 감사합니다.',
 '저희의 메뉴가 만족스러우셨다니 보람 찹니다 준비한 메뉴들이 칭찬 받을 때가 진짜 희열 넘치고 행복한 거 같아요칭찬만 받을 수 있게 매일 매장에서 연습 하고 있습니다다음에도 기회가 된다면 저희와 함께 즐거운 시간 보내시길 바라며, 오늘도 따뜻한 하루 되세요',
 '안녕하세요 1988응답하라추억의옛날도시락 입니다.',
 '오늘도 저희 매장에 귀한 발걸음을 해주셔서 영광입니다 맛에 대한 칭찬은 정말 언제 들어도 행복한 거 같아요 리뷰에서 이렇게 맛있다고 해주시는 걸 볼 때 마다, 다시 에너지가 모두 충전되는 느낌이더라고요 감사합니다  언제 어디 서든 기분 좋은 일들만 가득하시길 바라요!',
 '그 꿈을 향해 계속해서 앞으로 달려 갈 수 있게 도와 주셔서 감사해요항상 건강하시고 오늘도 행복한 하루 보내세요!',
 '행복한 하루를 만들어 주셔서 감사합니다와아!',
 '오늘도 행복한 하루 보내세요!',
 '저희 매장을 찾아주셔서 진심으로 감사드립니다저희는 항상 맛있는 맛만행복한 맛만 드리고 싶어요 찾아 주신 것만으로도 감사해서 보답 드리고 싶거든요 계속해서 지금보다 발전시켜 나갈게요!',
 '행복한 시간 되셨을까요?',
 '1988응답하라추억의옛날도시락을 선택해 주셔서 감사합니다.',
 '안녕하세요 1988응답하라추억의옛날도시락 입니다.',
 '1988응답하라추억의옛날도시락을 선택해 주셔서 감사합니다.',
 '1988응답하라추억의옛날도시락을 선택해 주셔서 감사합니다.',
 '저희 1988응답하라추억의옛날도시락을 찾아 주셔서 

In [96]:
len(store_sentence)

10890

In [97]:
pd.DataFrame(store_sentence) # 거의 복붙, 자동 생성 문장류. 역으로 인사말 등으로 구조 활용 가능

,0
0,"저희 매장을 찾아 주셔서 다시 한번 감사의 말씀드리며, 언제나 행복한 순간들만 가득..."
1,언제든지 행복한 배부름을 느끼게 해 드릴 수 있도록 저희가 있지 않겠습니까!
2,"언제나 웃음 가득한 일들만 계속되시길 바라며, 행복한 하루 보내세요"
3,앞으로도 맛의 성숙도를 높이기 위해 부던히 노력하겠습니다앞으로도 행복한 하루 보내시...
4,저희 1988응답하라추억의옛날도시락의 메뉴를 주문해 주셔서 감사합니다.
...,...
10885,안녕하세요귀한 식사에 보돌미역 수영점을찾아주셔서 정말 감사드립니다부모님께서 맛있...
10886,안녕하세요맛있게 드셨다니 정말 기쁘구요멋진 칭찬주셔서 넘넘 감사드립니다언제나 행복한...
10887,안녕하세요맛있는 식사되셨다니 넘넘 기쁩니다그리고 멋진 칭찬으로 격려해주셔서진심으로 ...
10888,찐맛탱집 홍대개미입니다 맛있게 드셔주셔서 항상 감사드립니다!


### 고객 관련 문장

- 'id' 포함 문장 : 1639개 -  고객 이름을 직접언급하는 경우는 거의 없고 id로 부름
  >> id 부분 제거해서 데이터로 활용 or 정제용으로 문장 전체 제거

- '고객님' 포함 문장: 10483개 - 넓은 의미 고객 문장으로 간주함
  >> 인사 및 감사말 다수, 데이터로 활용

In [98]:
customer_sentence_id = []

for review in yogiyo_reply_sentence:
    for sentence in review:  # enumerate를 사용하여 문장의 인덱스를 추적
        if '님' in sentence and '고객님' not in sentence:
          customer_sentence_id.append(sentence)


In [99]:
len(customer_sentence_id)  #5.6만개중. 매우 적은 편. 제거에 활용 가능

1639

In [100]:
customer_sentence_id

['앞으로는 기사님께 드리기 전에 한번 더 확인해서 이런일이 없도록 조치해 두겠습니다.',
 '다음에도 오늘처럼 빨리 받으실 수 있게 기사님들과 잘 소통해서 유지하는 방법으로 찾아보겠습니다오늘도 화이팅 넘치는 하루 보내시길 바라요!',
 '앞으로 기사님들께 메뉴 드리면서 요청사항 잘 지켜달라는 당부의 말씀까지 드릴 수 있도록 하겠습니다.',
 'al님  행복한 하루를 만들어 주셔서 감사합니다저희 매장의 메뉴를 즐겨 주셔서 감사합니다!',
 '안녕하세요 nh님  수 많은 매장들을 헤치고 와주셔서 감사합니다맛집 랭킹에 올라가자고 다짐하고 있었는데, 칭찬해 주시니 눈물이 나네요.',
 'cu님의 소중한 시간에 저희 매장이 함께 할 수 있어서 너무나도 영광이에요!',
 'ma님의 칭찬은 저를 춤추게 하네요 w 가족을 위해 준비하는 마음으로 늘 최고만 드리고 싶은 저희 매장의 마음 알랑가몰라행복한 순간엔 항상 저희를 떠올리실 수 있도록 더욱 더 노력하겠습니다 늘 건강하시고, 오늘도 행복한 하루 보내세요!',
 '그 마음으로 늘 준비하고있습니다인생 최고의 맛집이라고 항상 sa님께서 기억하시도록 항상 노력하겠습니다자주봬요 우리 언제나 웃음 가득한 일들만 계속되시길 바라며, 행복한 하루 보내세요',
 'wi님의 리뷰에 보답할 수 있도록, 저희 매장은 끊임없이 노력하도록 약속드립니다!',
 'ma님, 안녕하세요.',
 '그래도 이렇게 좋은 말씀 들을 수 있어서 너무 보람 찼습니다 te님 감사합니다하루하루 더 행복해 지셨음 좋겠습니다',
 "' 라는 감탄사가 바로 나왔으면 하는 마음으로 항상 노력하고 있답니다0 dl님께서 저희의 이 마음을 알아봐 주셔서 너무 보람차요마음속에 1988응답하라추억의옛날도시락이 새겨지실 수 있도록 노력 많이 해 보겠습니다",
 '소중한 리뷰 남겨 주셔서 감사합니다 te님주문 해 주셨을 때, 저희는 맛을 약속 드린다고 생각하고 있습니다  물론 기대하시는 맛을 매번 만족 드릴 순 없겠지만 최대한 노력하고 연구하려고 합니다.',
 '그래도 이렇게 좋은 말

- 짧은 문장인 경우 제외 필요

In [50]:
pd.DataFrame(customer_sentence)

""


### 칭찬 vs 클레임 관련 문장

In [106]:
remove_sentence = store_sentence + customer_sentence_id

- 전체 문장들에서 업체, 고객 관련 문장 제외함

In [107]:
content_sentence = [item for item in total_reply_sentence if item not in remove_sentence] # sum sentence 보다는 store sentence가 더 나음

In [108]:
pd.DataFrame(content_sentence)

,0
0,이렇게 저희 매장 찾아주셔서 감사드립니다!
1,"주방에서 힘든 일도 있고 늘 쉽지만은 않지만, 이런 리뷰 읽을 때 마다 보람차고 다..."
2,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다.
3,저희를 찾아 주신 만큼 100 만족감을 드리고 싶었는데 그러지 못한 것 같아 너무 ...
4,수 많은 매장들을 헤치고 와주셔서 감사합니다!
...,...
36505,다음에도 이용해주세용
36506,감사합니다!
36507,다음에도 이용해주세용
36508,감사합니다!


## 최종 데이터 - content sentence로 한정

In [111]:
len(total_reply_sentence), len(store_sentence + customer_sentence_id), len(content_sentence)

(56141, 23012, 36510)

In [112]:
content_sentence

['이렇게 저희 매장 찾아주셔서 감사드립니다!',
 '주방에서 힘든 일도 있고 늘 쉽지만은 않지만, 이런 리뷰 읽을 때 마다 보람차고 다시 시작할 수 있는 거 같습니다 정말 감사드려요오늘도 화이팅 있게 마무리하시길 바라요',
 '소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다.',
 '저희를 찾아 주신 만큼 100 만족감을 드리고 싶었는데 그러지 못한 것 같아 너무 죄송합니다 말씀 주신 부분에 대해서는 주의하여 다음 주문시에는 실수 없도록 꼼꼼하게 확인하겠습니다.',
 '수 많은 매장들을 헤치고 와주셔서 감사합니다!',
 '저희 매장의 메뉴를 제대로 즐기신 거 같아 너무 영광이에요 모두 마음에 드셨을까요?',
 '전부 정성껏 준비했지만, 늘 주문해 주신 분들의 평가를 기다릴 땐 떨려요더욱 만족스러운 시간 드릴 수 있게 노력하겠습니다',
 '수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 성공이였을까요?',
 '늘 어떻게하면 더 맛있게 드릴 수 있을까 고민 많이 하고 있어요항상 노력하고 발전하도록 하겠습니다항상 같은 만족감 드릴 수 있도록 언제나 최선을 다해 운영 하겠습니다',
 '저희 매장 찾아주신 것도 감사한데 리뷰까지.',
 '매번 찾아 주셔도 매번 시켜 먹고 싶은 매장이 되겠습니다 하시는 일 모두 번창하시길 진심으로 바라고 있을게요',
 '다음에 또 만나요!',
 '저희 매장 다시 와주실꺼죠?',
 '언제든 신호만 주시면 바로 달려가도록 하겠습니다!',
 '다음에 또 만나요!',
 '귀한 발걸음 감사합니다!',
 '아낌없이 칭찬해 주셔서 기분이 좋습니다 저희 매장은 맛과 양, 서비스 등 모든 면에서 최고가 되기 위해 항상 최선을 다하고 있습니다!',
 '또다시 저희 매장이 생각나실 때 만날 수 있길 바라고 있을게요',
 '귀한 발걸음 감사합니다!',
 '저희 매장과 즐거운 시간 보내셨을까요?',
 '칭찬 감사합니다 늘 최선을 다해서 조리하고 있지만, 이렇게 맛있게 드셔 주셨다고 해주시는 리뷰 보면 정말 마음이 따

In [113]:
yogiyo_reply_sentence

[['이렇게 저희 매장 찾아주셔서 감사드립니다!',
  '맛 뿐만 아니라 양도, 서비스도 모두 최상의 품질을 드리려고 노력하고 있습니다 모든 직원들이 힘내서 고객님의 마음을 잡기 위해 계속 노력하겠습니다항상 건강 챙기시고, 오늘도 즐거운 하루 보내시길 바라겠습니다'],
 ['좋은 리뷰 덕분에 기분이 행복해지는 것 같아요고객님께서 맛있다고 말씀 주시니 피로가 싹 사라지네요!',
  '주방에서 힘든 일도 있고 늘 쉽지만은 않지만, 이런 리뷰 읽을 때 마다 보람차고 다시 시작할 수 있는 거 같습니다 정말 감사드려요오늘도 화이팅 있게 마무리하시길 바라요'],
 ['소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다.',
  '저희를 찾아 주신 만큼 100 만족감을 드리고 싶었는데 그러지 못한 것 같아 너무 죄송합니다 말씀 주신 부분에 대해서는 주의하여 다음 주문시에는 실수 없도록 꼼꼼하게 확인하겠습니다.',
  '저희 매장을 찾아 주셔서 다시 한번 감사의 말씀드리며, 언제나 행복한 순간들만 가득하시길 바라겠습니다!'],
 ['수 많은 매장들을 헤치고 와주셔서 감사합니다!',
  '저희 매장의 메뉴를 제대로 즐기신 거 같아 너무 영광이에요 모두 마음에 드셨을까요?',
  '전부 정성껏 준비했지만, 늘 주문해 주신 분들의 평가를 기다릴 땐 떨려요더욱 만족스러운 시간 드릴 수 있게 노력하겠습니다'],
 ['수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 성공이였을까요?',
  '저희 메뉴 칭찬해 주셔서 감사합니다 고객님!',
  '늘 어떻게하면 더 맛있게 드릴 수 있을까 고민 많이 하고 있어요항상 노력하고 발전하도록 하겠습니다항상 같은 만족감 드릴 수 있도록 언제나 최선을 다해 운영 하겠습니다'],
 ['저희 매장 찾아주신 것도 감사한데 리뷰까지.',
  '감동입니다고객님들의 배고픔을 저희는 지켜만 볼 수 없습니다!',
  '언제든지 행복한 배부름을 느끼게 해 드릴 수 있도록 저희가 있지 않겠습니까!',
  '매번 찾아 주셔도 매번 

In [114]:
# yogiyo_content_reviews = []
# indices = []

# for idx, review_sentences in enumerate(yogiyo_reviews_sentence):
#     for sentence in review_sentences:
#         if sentence in content_sentence:  # content_sentence에 있는 문장인 경우
#             yogiyo_content_reviews.append(review_sentences)  # 전체 리뷰를 추가합니다
#             indices.append(idx)  # 리뷰의 인덱스를 추가합니다
#             break  # 이미 찾았으므로 이 리뷰는 더 이상 검사하지 않습니다

yogiyo_content_reviews = []
indices = []

for idx, review_sentences in enumerate(yogiyo_reply_sentence):
    filtered_review = []
    for sentence in review_sentences:
        if sentence in content_sentence:  # content_sentence에 있는 문장인 경우
            filtered_review.append(sentence)  # content_sentence에 있는 문장만 추가합니다
    if filtered_review:  # 적어도 하나의 content_sentence가 있는 경우에만 처리합니다
        yogiyo_content_reviews.append(filtered_review)  # 전체 리뷰를 추가합니다
        indices.append(idx)  # 리뷰의 인덱스를 추가합니다



In [124]:
len(yogiyo_content_reviews) # 총 1.8만개 중에서 3000개 리뷰 정도는 사라짐

15200

In [125]:
yogiyo_content_reviews


[['이렇게 저희 매장 찾아주셔서 감사드립니다!'],
 ['주방에서 힘든 일도 있고 늘 쉽지만은 않지만, 이런 리뷰 읽을 때 마다 보람차고 다시 시작할 수 있는 거 같습니다 정말 감사드려요오늘도 화이팅 있게 마무리하시길 바라요'],
 ['소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다.',
  '저희를 찾아 주신 만큼 100 만족감을 드리고 싶었는데 그러지 못한 것 같아 너무 죄송합니다 말씀 주신 부분에 대해서는 주의하여 다음 주문시에는 실수 없도록 꼼꼼하게 확인하겠습니다.'],
 ['수 많은 매장들을 헤치고 와주셔서 감사합니다!',
  '저희 매장의 메뉴를 제대로 즐기신 거 같아 너무 영광이에요 모두 마음에 드셨을까요?',
  '전부 정성껏 준비했지만, 늘 주문해 주신 분들의 평가를 기다릴 땐 떨려요더욱 만족스러운 시간 드릴 수 있게 노력하겠습니다'],
 ['수 많은 매장들을 헤치고 와주셔서 감사합니다메뉴 그 자체로 행복을 드리고 싶었는데 성공이였을까요?',
  '늘 어떻게하면 더 맛있게 드릴 수 있을까 고민 많이 하고 있어요항상 노력하고 발전하도록 하겠습니다항상 같은 만족감 드릴 수 있도록 언제나 최선을 다해 운영 하겠습니다'],
 ['저희 매장 찾아주신 것도 감사한데 리뷰까지.',
  '매번 찾아 주셔도 매번 시켜 먹고 싶은 매장이 되겠습니다 하시는 일 모두 번창하시길 진심으로 바라고 있을게요'],
 ['다음에 또 만나요!'],
 ['저희 매장 다시 와주실꺼죠?', '언제든 신호만 주시면 바로 달려가도록 하겠습니다!', '다음에 또 만나요!'],
 ['귀한 발걸음 감사합니다!',
  '아낌없이 칭찬해 주셔서 기분이 좋습니다 저희 매장은 맛과 양, 서비스 등 모든 면에서 최고가 되기 위해 항상 최선을 다하고 있습니다!',
  '또다시 저희 매장이 생각나실 때 만날 수 있길 바라고 있을게요'],
 ['귀한 발걸음 감사합니다!'],
 ['저희 매장과 즐거운 시간 보내셨을까요?',
  '칭찬 감사합니다 늘 최선을 다해서 조리하고 있지만, 이렇게 맛있게 

In [126]:
len(indices)

15200

In [127]:
indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 22,
 23,
 24,
 25,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 113,
 114,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,


In [128]:
content_df = yogiyo.iloc[indices]

In [120]:
content_shuffled_df = content_df.sample(frac=1, random_state=42)

content_shuffled_df.to_csv(f'{DATA_PATH}content_shuffled_df.csv', index = False)

In [121]:
content_shuffled_df.shape

(15200, 2)

# 모델 학습

In [65]:
content_shuffled_df

,review,owner_reply
3127,적게 일하고 많이 버세요 사장님,안녕하세요 소중한 리뷰 남겨 주셔서 감사합니다! 칭찬이 사람을 기쁘게 한다고 하던데...
4763,좋아용!! 배달도 빠르고 맛도 좋아요,감사해요! 단체주문도 항상 열려있으니 저희 '사소우' 생각나실때 언제든 찾아주세요 ...
7070,배송빠르고 반찬이 모두 집에서 먹던 그맛 입니다 오랫만에 맛있게 먹었습니다,주문 감사드립니다 언제나 가족이 먹는다는 생각으로 만들고 있답니다맛있게 드시고 기...
7959,쫄면맛집이라고 소문나서 먹었는데 진짜 쫄면맛집이 맞습니다 김밥도 둘다 넘 맛있게 ...,안녕하세요 so님쫄면 맛집으로 소문이 났나요? 보람있는 순간입니다앞으로도 맛...
8201,맛있어요,08님인기메뉴 싱글세트 울 고객님들이 제일 많이 찾으시는 메뉴랍니다. 맛있어요 이 ...
...,...,...
10768,맛있게 잘 먹었습니다,맛있게 드셔주신 덕분에 일하며 쌓인 피로가한방에 날아가는 것 같습니다!!항상 철저한...
7266,배달도 빠르고 정말맛있었어요!! 다만 문앞에놔주시고 문자달라고 부탁드렸는데..벨을누...,글쎄요 요청사항 저도 본것 같은데 기사님은 제대로 보질 못했나 보네요.기사님들...
7459,아주 맛나게 먹었어요 맵기도 딱 좋아요,소중한 리뷰 감사합니다맛있게 드시고 고맙습니다더'노력 하겠습니다새해 복많이 받으세요
7888,늦은 시간인데 배달돼서 감사해요번창하세요,안녕하세요 so님저희가 해드릴수 있는건 뭐든 해드려야지요고맙습니다


## 데이터 학습

In [66]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00


In [129]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name = "skt/kogpt2-base-v2"
model_name = "EasthShin/Youth_Chatbot_Kogpt2-base"

model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, #해당 모델의 토크나이저
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

In [130]:
# sample_df = shuffled_df.iloc[:3000]
# sample_df = food_shuffled_df_1.iloc[:3000]
# sample_df = food_shuffled_df_2_top5.iloc[:3000]
sample_df = content_shuffled_df.iloc[:3000]


In [140]:
sample_df

,review,owner_reply
3942,오늘도맛있게해주셔서감사합니다,"m0님,안녕하세요 시간내어 리뷰작성해주셔서 정말 감사합니다항상 고객님들 주문 하나하..."
4667,너무 맛있어요 감사합니다,저희 매장 찾아주신 것도 감사한데 리뷰까지..감동입니다계속해서 더 많은 기대 부탁드...
4259,먹다가 정신이 없어서 2조각 순삭하고 찍었네요,식사는 맛있게 하셨나요? 혜화 대학로 SNS피자랭킹맛집피자보이시나 대학로점을 믿고주...
12733,맛있어요. 기존에 시켜먹던데 버리고 .여기서만 시키게요,만족해주신 그 맛! 절대 변하지 않고 앞으로도 쭉 만족하실 수 있도록 최선을 다하겠...
9632,주말저녁 맛있게먹었어요,안녕하세요 고객님 D번거로우실텐데 리뷰 작성해 주셔서 감사합니다!앞으로도 맛있게 ...
...,...,...
4681,너무 맛있었오요,5점 짜리 리뷰로 이런 따뜻함을 안고 와 주셔서 감사해요오예w 우리 고객님께 인정받...
17999,맛있게 잘 먹었습니다. 최고!!!,최고라해주시니 너무나도 감사한일이네요 너무 덥네요 시윈한 주말잘 보내셔요
1075,맛있는데 다른곳 보다 안 매워서 다음에는 더 맵게 먹어볼께요,맛있게 드셔주셔서 감사합니다요청사항은 조리전 늘 꼼꼼하게 체크하고 있으니 필요하신 ...
17066,치즈 추가안해도 충분했어요넘 맛있게 먹었습니다!,안녕하세요 프라텔로입니다!저희 프라텔로를 이용해주셔서 정말 감사드립니다!앞으로도 ...


In [132]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.question = df["review"].tolist()
        self.answer = df["owner_reply"].tolist()
    def __len__(self):
        return len(self.question)

    def __getitem__(self,idx):
        return "<q>" + self.question[idx] + "</s><a>" + self.answer[idx] + "</s>"

In [ ]:
# class ChatDataset(torch.utils.data.Dataset):
#     def __init__(self,df):
#         self.review = df["review"].tolist()
#         self.reply = df["reply"].tolist()
#         self.store = df["store"].tolist()

#     def __len__(self):
#         return len(self.review)

#     def __getitem__(self,idx):
#         return "<w>" + self.store[idx] + "</s><q>" + self.review[idx] + "</s><a>" + self.reply[idx] + "</s>"
#                 # 임의의 sos 토큰 <q>와 eos 토큰 </s> 삽입

In [71]:
# class ChatDataset(torch.utils.data.Dataset):
#     def __init__(self,df):
#         self.review = df["review"].tolist()
#         self.reply = df["reply"].tolist()
#         self.store = df["store"].tolist()

#     def __len__(self):
#         return len(self.review)

#     def __getitem__(self,idx):
#         return "<w>" + self.store[idx] + "</s><q>" + self.review[idx] + "</s><a>" + self.reply[idx] + "</s>"
                # 임의의 sos 토큰 <q>와 eos 토큰 </s> 삽입

In [133]:
def collate_fn(batch): #  배치 데이터를 전처리(형식변환) # 데이터 길이 맞추기 위해 collate(모으다, 조립하다)
    x = tokenizer(batch, return_tensors="pt",padding=True)
    return {"x":x}

# return_tensors="pt" 토큰화된 결과를 PyTorch의 텐서 형식으로 반환 (Tensorflow('tf'), Numpy형식 가능('np'))

In [134]:
dt = ChatDataset(sample_df)
dl = torch.utils.data.DataLoader(dt,batch_size=2,collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   455,   405, 30899,  7235,  7496, 18479, 15084,  7810,  7788,
          21868,  8708,  7172,  7182,     1,  9724,   439,   405,   451,   391,
           7177,   387,  7967,  7114,  8702,  7801,  8084, 10135, 16977,  9664,
           7685, 43131, 15084,  7810,  7788, 29205, 15940,  8708,  7172,  7182,
           8710,  7767, 12298,  7177,  7285, 16812, 25926,  8022, 36654, 17930,
          20129, 26616, 16691, 18882,  9535,  7892,  7847, 11608,  9546,  7692,
          12247, 14337, 15084,  7801,  8091,   376,   376, 15940,  8708,  7172,
           7182, 33753,  7204,  6951,  7556,  7442, 20479, 18429,   445, 13119,
           8711,  7281,  9885,  8155,  8140, 12521,  9564, 22412, 15084,  7801,
           8084,     1,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     

In [135]:
model(**batch["x"]).logits.shape

torch.Size([2, 140, 51200])

In [136]:
tokenizer.pad_token_id

3

In [137]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):
        x = batch["x"].to(device)
        pred = model(**x).logits
        n_class = pred.shape[-1]
        pred = pred[:,:-1]
        pred = pred.reshape(-1,n_class)

        tgt = x["input_ids"][:,1:]
        tgt = tgt.flatten()

        mask = tgt != 3
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred,tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [138]:
batch_size = 2
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 10

- 하나의 모델 점수 보기

In [139]:
reset_seeds(SEED)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=3e-5)

train_dt = ChatDataset(sample_df) # 데이터셋!!! (sample df)
train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

loss_list = []
for _ in tqdm(range(epochs)):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    loss_list.append(train_loss)

    print(train_loss)



  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

2.2750345913867154


  0%|          | 0/1500 [00:00<?, ?it/s]

1.5489916837314766


...

0.5226818387111029


  0%|          | 0/1500 [00:00<?, ?it/s]

0.44558923777441184


  0%|          | 0/1500 [00:00<?, ?it/s]

0.38628139880299567


In [142]:
score = pd.DataFrame(loss_list)
score.columns = ['ygy_cont3000_epoch10']

score
# new_df = pd.DataFrame(loss_list)
# new_df.columns = ['']

# df = pd.concat([df,new_df],axis=1)
# df

,ygy_cont3000_epoch10
0,2.275035
1,1.548992
2,1.276097
3,1.075967
4,0.893973
5,0.744478
6,0.622660
7,0.522682
8,0.445589
9,0.386281


- 모델 저장

In [144]:
# model.save_pretrained(f"{DATA_PATH}kogpt2_chat1")
# model.save_pretrained(f"{DATA_PATH}kogpt2_chat2_food3000")
# model.save_pretrained(f"{DATA_PATH}kogpt2_chat3_food_top5_3000")

model.save_pretrained(f"{DATA_PATH}ygy_cont3000_youth_ver1")



- 모델별로 점수 담기 --> 다른 모델들 GPU 오류로 불가

In [82]:
# import torch
# from transformers import AutoModelForCausalLM
# from tqdm import tqdm
# import pandas as pd

# # model_name = "skt/kogpt2-base-v2"  # 저장
# # # model_name = 'EleutherAI/polyglot-ko-5.8b' # 메모리 오류

# # # model_name = 'beomi/KoAlpaca-Polyglot-5.8B'
# # # model_name = 'nlpai-lab/kullm-polyglot-5.8b-v2'


# # 모델명과 학습 관련 파라미터들 설정
# model_name = 'skt/kogpt2-base-v2'  # 사전학습 모델의 이름
# epochs = 10
# batch_size = 2

# # 결과를 저장할 리스트 초기화
# epoch_scores = []

# # 시드 초기화 및 모델 생성
# reset_seeds(SEED)
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

# train_dt = ChatDataset(sample_df)
# train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# # 학습 루프 실행
# loss_list = []
# for epoch in tqdm(range(epochs)):
#     train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
#     loss_list.append(train_loss)

#     # 여기서 원하는 점수를 얻는 코드를 실행하고 epoch_scores 리스트에 추가
#     # 예를 들어, 테스트 데이터로 모델을 평가하여 점수를 계산하고 추가할 수 있습니다.
#     test_score = evaluate_model(model, test_dl)  # 적절한 함수로 평가 점수를 얻음
#     epoch_scores.append(test_score)

#     print(f"Epoch [{epoch + 1}/{epochs}] - Train Loss: {train_loss:.4f} - Test Score: {test_score:.4f}")

# # 결과를 데이터프레임으로 저장
# result_df = pd.DataFrame({'Epoch': range(1, epochs + 1), 'Test Score': epoch_scores})

# # 결과 출력
# print(result_df)


In [83]:
# model.save_pretrained(f"{DATA_PATH}kogpt2_chat1") :





# model.save_pretrained(f"{DATA_PATH}EleutherAI_chat1")

# tokenizer.save_pretrained(f"{DATA_PATH}kogpt2_chat1_tokenizer")